# Import Library

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE 

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score, f1_score, precision_score

# Load dataset

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/willyrizkiyan/Classification-Churn-Telco/main/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


# Data Preprocessing

In [7]:
# Change TotalCharges column type to float
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

In [8]:
# recheck info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Observation:
* After change `TotalCharges` column type to float64, turns out actually there are 11 missing values.

In [9]:
df[df.isna().any(axis=1)]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


Observation:
* Customers with null in `TotalCharges` are customers who have `Tenure` = 0. It's possible that they are new customers.
Instead of dropping them out, we can change null value in `TotalCharges` to 0.
* Column name for `customerID`, `gender` and `tenure` will be changed into proper case.

In [10]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [11]:
df = df.rename(columns={"customerID": "CustomerID", "gender": "Gender", "tenure":"Tenure"})

## Check Missing Value

In [12]:
df.isna().sum()

CustomerID          0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
Tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

There is no missing values

## Check Duplicated Value

In [13]:
df.duplicated().sum()

0

There is no duplicated values

## Check Data for Encoding

In [14]:
# Separate columns into numerical and categorical
all_columns = df.columns.to_list()
numerical = ['SeniorCitizen', 'Tenure', 'MonthlyCharges', 'TotalCharges']
categorical = list(set(all_columns) - set(numerical))

In [15]:
# check value counts and odd values in categorical columns, such as string "-", or other
for i in df[categorical].columns:
    x = df[categorical][i].value_counts()
    print(i)
    print(x,'\n')

CustomerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: CustomerID, Length: 7043, dtype: int64 

Churn
No     5174
Yes    1869
Name: Churn, dtype: int64 

StreamingMovies
No                     2785
Yes                    2732
No internet service    1526
Name: StreamingMovies, dtype: int64 

Partner
No     3641
Yes    3402
Name: Partner, dtype: int64 

Contract
Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64 

Dependents
No     4933
Yes    2110
Name: Dependents, dtype: int64 

OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64 

PhoneService
Yes    6361
No      682
Name: PhoneService, dtype: int64 

PaymentMethod
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card

**Observation :**

**Columns thas will use Label Encoding (2 values)**
* PhoneService, Churn, Partner, Dependents, PaperlessBilling, Gender

**Columns that needs to reprocess for Label Encoding (Yes, No, No internet service, No phone service). No internet service and No phone service are actually "No"**
* TechSupport, StreamingTV, OnlineSecurity, StreamingMovies, MultipleLines, DeviceProtection, OnlineBackup

**Columns that will be processed using One Hot Encoding (more than 2 values)**
* InternetService, Contract, PaymentMethod

**Column that will be dropped**
* customerID


## Change String For Label Encoding

In [16]:
cols_2 = ['PhoneService', 'Churn', 'Partner', 'Dependents', 'PaperlessBilling', 'Gender']
cols_more_than_2 = ['TechSupport', 'StreamingTV', 'OnlineSecurity', 'StreamingMovies', 'MultipleLines', 'DeviceProtection', 'OnlineBackup']
df[cols_more_than_2] = df[cols_more_than_2].replace({'No internet service':'No', 'No phone service':'No'})

cols = cols_2 + cols_more_than_2

## Label Encoding

In [17]:
le = preprocessing.LabelEncoder()

for column in cols:
  le.fit(df[column])
  df[column] = le.transform(df[column])

df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,0,0,2,1,0,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,0,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   int64  
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   int64  
 4   Dependents        7043 non-null   int64  
 5   Tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   int64  
 7   MultipleLines     7043 non-null   int64  
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   int64  
 10  OnlineBackup      7043 non-null   int64  
 11  DeviceProtection  7043 non-null   int64  
 12  TechSupport       7043 non-null   int64  
 13  StreamingTV       7043 non-null   int64  
 14  StreamingMovies   7043 non-null   int64  
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   int64  


## One Hot Encoding

In [19]:
#dummies_internetservice = pd.get_dummies(df['InternetService'],prefix='is')
#dummies_internetservice.head()
cols_one_hot = ['InternetService', 'Contract', 'PaymentMethod']

df = pd.get_dummies(data=df, columns=cols_one_hot)
df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,0,1,0,1,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,1,0,0,0,34,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,1,0,0,0,2,1,0,1,1,...,1,0,0,1,0,0,0,0,0,1
3,7795-CFOCW,1,0,0,0,45,0,0,1,0,...,1,0,0,0,1,0,1,0,0,0
4,9237-HQITU,0,0,0,0,2,1,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   CustomerID                               7043 non-null   object 
 1   Gender                                   7043 non-null   int64  
 2   SeniorCitizen                            7043 non-null   int64  
 3   Partner                                  7043 non-null   int64  
 4   Dependents                               7043 non-null   int64  
 5   Tenure                                   7043 non-null   int64  
 6   PhoneService                             7043 non-null   int64  
 7   MultipleLines                            7043 non-null   int64  
 8   OnlineSecurity                           7043 non-null   int64  
 9   OnlineBackup                             7043 non-null   int64  
 10  DeviceProtection                         7043 no

## Drop Unnecessary Column

In [21]:
df = df.drop(['CustomerID'], axis=1)
df.head()

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,1,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,1,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,1,0,0,0,45,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,0,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Gender                                   7043 non-null   int64  
 1   SeniorCitizen                            7043 non-null   int64  
 2   Partner                                  7043 non-null   int64  
 3   Dependents                               7043 non-null   int64  
 4   Tenure                                   7043 non-null   int64  
 5   PhoneService                             7043 non-null   int64  
 6   MultipleLines                            7043 non-null   int64  
 7   OnlineSecurity                           7043 non-null   int64  
 8   OnlineBackup                             7043 non-null   int64  
 9   DeviceProtection                         7043 non-null   int64  
 10  TechSupport                              7043 no

# Imbalanced Data Handling

In [23]:
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [24]:
from imblearn import under_sampling, over_sampling

X = df.drop(['Churn'],axis=1)
y = df['Churn']

# Undersampling
X_under, y_under = under_sampling.RandomUnderSampler(random_state=42).fit_resample(X, y)

# Oversampling with smote
X_over_smote, y_over_smote = over_sampling.SMOTE(random_state=42).fit_resample(X, y)

In [25]:
df_undersampling = pd.concat([X_under, y_under], axis=1)
df_undersampling['Churn'].value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

In [26]:
df_oversampling_smote = pd.concat([X_over_smote, y_over_smote], axis=1)
df_oversampling_smote['Churn'].value_counts()

0    5174
1    5174
Name: Churn, dtype: int64

# Modelling

## Split Dataset

In [27]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## K-Nearest Neighbor

In [29]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

## Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)
y_pred_reg = logreg.predict(X_test)

## Decision Tree

In [31]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred_dt = dt.predict(X_test)

## Random Forest with Balanced Data

In [32]:
X_us = df_undersampling.drop('Churn', axis=1)
y_us = df_undersampling['Churn']

X_sm = df_oversampling_smote.drop('Churn', axis=1)
y_sm = df_oversampling_smote['Churn']

In [33]:
X_train_us, X_test_us, y_train_us, y_test_us = train_test_split(X_us, y_us, test_size=0.25, random_state=42)
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

In [34]:
rf_us = RandomForestClassifier(random_state=42)
rf_us.fit(X_train_us, y_train_us)
y_pred_us = rf_us.predict(X_test_us)

In [35]:
rf_sm = RandomForestClassifier(random_state=42)
rf_sm.fit(X_train_sm, y_train_sm)
y_pred_sm = rf_sm.predict(X_test_sm)

## Random Forest with Imbalanced Data

In [36]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [37]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.25, random_state=42)

In [38]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

In [39]:
cm = confusion_matrix(y_test_rf, y_pred_rf)
cm

array([[1166,  116],
       [ 258,  221]])

## XGBoost

In [40]:
from xgboost import XGBClassifier

X = df.drop('Churn', axis=1)
y = df['Churn']

In [41]:
X_train_xg, X_test_xg, y_train_xg, y_test_xg = train_test_split(X, y, test_size=0.25, random_state=42)

In [42]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train_xg, y_train_xg)
y_pred_xg = xgb.predict(X_test_xg)

In [43]:
cm = confusion_matrix(y_test_xg, y_pred_xg)
cm

array([[1165,  117],
       [ 228,  251]])

# Metric Evaluation

Matrix Accuracy :
* Over 0.9 --> Very Good
* 0.7 - 0.9 --> Good
* 0.6 - 0.7 --> OK
* Below 0.6 --> Poor

Matrix AUC :
* Over 0.9 --> Very Good
* 0.8 - 0.9 --> Good
* 0.7 - 0.8 --> Fair
* 0.6 - 0.7 --> Poor
* Below 0.6 --> Not Good

Matrix F1 Score :
* Over 0.9 --> Very Good
* 0.8 - 0.9 --> Good
* 0.5 - 0.8 --> OK
* Below 0.5 --> Not Good

In [44]:
print('Akurasi KNN :',accuracy_score(y_test, y_pred_knn))
print('Akurasi Logreg :',accuracy_score(y_test, y_pred_reg))
print('Akurasi Decision Tree :',accuracy_score(y_test, y_pred_dt))
print('Akurasi Random Forest Undersampling :',accuracy_score(y_test_us, y_pred_us))
print('Akurasi Random Forest Oversampling Smote :',accuracy_score(y_test_sm, y_pred_sm))
print('\n')
print('AUC Random Forest Imbalanced Data :',roc_auc_score(y_test_rf, y_pred_rf))
print('F1 Score Random Forest Imbalanced Data :',f1_score(y_test_rf, y_pred_rf))
print('Precision Random Forest Imbalanced Data :',precision_score(y_test_rf, y_pred_rf))
print('Recall Random Forest Imbalanced Data :',recall_score(y_test_rf, y_pred_rf))
print('\n')
print('AUC XGBoost :',roc_auc_score(y_test_xg, y_pred_xg))
print('F1 Score XGBoost :',f1_score(y_test_xg, y_pred_xg))
print('Precision XGBoost :',precision_score(y_test_xg, y_pred_xg))
print('Recall XGBoost :',recall_score(y_test_xg, y_pred_xg))

Akurasi KNN : 0.7694491766042022
Akurasi Logreg : 0.8165814877910278
Akurasi Decision Tree : 0.7296990346394094
Akurasi Random Forest Undersampling : 0.7518716577540107
Akurasi Random Forest Oversampling Smote : 0.8542713567839196


AUC Random Forest Imbalanced Data : 0.6854471256094502
F1 Score Random Forest Imbalanced Data : 0.5416666666666666
Precision Random Forest Imbalanced Data : 0.655786350148368
Recall Random Forest Imbalanced Data : 0.4613778705636743


AUC XGBoost : 0.7163723500923336
F1 Score XGBoost : 0.5926800472255017
Precision XGBoost : 0.6820652173913043
Recall XGBoost : 0.524008350730689


# Summary

Based on matrix, highest accuracy using random forest oversampling smote is good accuracy.

AUC for random forest imbalanced data is poor and F1-Score is OK.

AUC for XGBoost is fair and F1-Score is OK.

For summary, random forest oversampling smote is best model to choose.
